# 第7章: 例の使用（Few-Shot プロンプティング）

- [レッスン](#レッスン)
- [演習](#演習)
- [サンプルプレイグラウンド](#サンプルプレイグラウンド)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立します。

In [1]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
import json

# utilsパッケージからhintsモジュールをインポート
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt, system='', prefill=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages":[
              {"role": "user", "content": prompt},
              {"role": "assistant", "content": prefill}
            ],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## レッスン

**Claudeにどのように振る舞ってほしいか（あるいはどのように振る舞ってほしくないか）の例を示すことは、以下の点で非常に効果的です**：
- 正しい答えを得る
- 正しい形式で答えを得る

このようなプロンプティングは「**few-shot プロンプティング**」とも呼ばれます。「ゼロショット」や「nショット」、「ワンショット」という言葉を耳にすることもあるかもしれません。「ショット」の数は、プロンプト内で使用される例の数を指します。

### 例

子供からの質問に答える「親ボット」を作ろうとしている開発者だと想像してください。**Claudeのデフォルトの応答はかなり形式的で機械的です**。これでは子供の心が傷つきます。

In [2]:
# プロンプト
PROMPT = "サンタさんはクリスマスにプレゼントを持ってきてくれますか？"

# Claudeの応答を表示
print(get_completion(PROMPT))

望むトーンを説明するのに時間をかけることもできますが、**理想的な応答の例をいくつかClaudeに与える**方がはるかに簡単です。

In [3]:
# プロンプト
PROMPT = """「A」として話す次の行を書いて、会話を完成させてください。
Q: 歯の妖精は本当にいるの？
A: もちろんだよ、かわいい子。歯を包んで今夜枕の下に置いてね。朝になったら何か待っているかもしれないよ。
Q: サンタさんはクリスマスにプレゼントを持ってきてくれますか？"""

# Claudeの応答を表示
print(get_completion(PROMPT))

以下のフォーマット例では、名前と職業を抽出し、それらを私たちが望む通りに正確にフォーマットする方法について、一連のフォーマット指示をステップバイステップでClaudeに説明することもできますが、**正しくフォーマットされた例をいくつかClaudeに提供すれば、そこから推論することができます**。`assistant`ターンの`<individuals>`に注目してください。これはClaudeを正しいスタートに導くためのものです。

In [ ]:
# 変数コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = """シルバーミスト・ホローは魅力的な村で、並外れた個性を持つ人々の住処でした。
その中にはリアム・パテル博士がいました。彼は地域の医療センターで外科手術の技術を革新した脳神経外科医でした。
オリビア・チェンは革新的な建築家で、持続可能で息をのむような設計で村の景観を変えました。
地元の劇場では、プロの訓練を受けた音楽家であり作曲家であるイーサン・コバックスの魅惑的な交響曲が演奏されていました。
地元の食材に情熱を注ぐ独学のシェフ、イザベラ・トーレスは、ファームトゥテーブルのレストランで大人気の料理を生み出し、食通にとって必須の訪問先となりました。
これらの素晴らしい個性を持つ人々は、それぞれ独特の才能を持ち、シルバーミスト・ホローの生活の鮮やかなタペストリーに貢献しました。
<individuals>
1. リアム・パテル博士 [脳神経外科医]
2. オリビア・チェン [建築家]
3. イーサン・コバックス [音楽家および作曲家]
4. イザベラ・トーレス [シェフ]
</individuals>

町の中心部では、シェフのオリバー・ハミルトンがファームトゥテーブルのレストラン、グリーン・プレートで料理シーンを変革しました。地元の有機食材にこだわるオリバーの姿勢は、食評家や地元の人々から絶賛を浴びています。
通りを少し下ったところには、リバーサイド・グローブ図書館があります。ここで主任司書のエリザベス・チェンは、すべての人にとって居心地の良い包括的な空間を作るために熱心に働いてきました。図書館の蔵書を増やし、子供向けの読書プログラムを確立する彼女の努力は、町の識字率に大きな影響を与えました。
魅力的な町の広場を歩くと、壁を飾る美しい壁画に魅了されるでしょう。これらの傑作は、有名なアーティスト、イザベラ・トーレスの作品です。彼女のリバーサイド・グローブの本質を捉える才能は、町に生命を吹き込みました。
リバーサイド・グローブの運動競技の成果も注目に値します。これは元オリンピック選手からコーチに転身したマーカス・ジェンキンスのおかげです。マーカスは自身の経験と情熱を活かして町の若者を指導し、リバーサイド・グローブ水泳チームを何度も地域選手権に導きました。
<individuals>
1. オリバー・ハミルトン [シェフ]
2. エリザベス・チェン [司書]
3. イザベラ・トーレス [アーティスト]
4. マーカス・ジェンキンス [コーチ]
</individuals>

オーク・バレーは魅力的な小さな町で、その技術と献身によってコミュニティに長続きする影響を与えた3人の注目すべき個人の住処です。
町の賑やかなファーマーズマーケットでは、情熱的な有機農家のローラ・シモンズを見つけることができます。彼女は美味しくて持続可能な方法で栽培された農産物で知られています。健康的な食生活を推進する彼女の献身は、町がよりエコ意識の高いライフスタイルを受け入れるよう促しました。
オーク・バレーのコミュニティセンターでは、熟練したダンスインストラクターのケビン・アルバレスが、あらゆる年齢層の人々に動きの喜びをもたらしました。彼の包括的なダンスクラスは、住民の間に団結と自己表現の感覚を育み、地元のアートシーンを豊かにしました。
最後に、tireless なボランティアのレイチェル・オコナーは、様々な慈善活動に時間を捧げています。他者の生活を改善することへの彼女のコミットメントは、オーク・バレー内に強いコミュニティ意識を作り出すのに不可欠でした。
それぞれのユニークな才能と揺るぎない献身を通じて、ローラ、ケビン、レイチェルはオーク・バレーの織物に自らを織り込み、活気に満ちた繁栄する小さな町を作り出すのを助けました。"""

# Claudeの応答の事前入力
PREFILL = "<individuals>"

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print("ユーザーターン:")
print(PROMPT)
print("\nアシスタントターン:")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

上記のコンテンツを変更せずにレッスンのプロンプトを試してみたい場合は、レッスンノートブックの一番下までスクロールして[**サンプルプレイグラウンド**](#サンプルプレイグラウンド)をご覧ください。

---

## 演習
- [演習 7.1 - 例によるメールのフォーマット](#演習-71---例によるメールのフォーマット)

### 演習 7.1 - 例によるメールのフォーマット
演習6.2をやり直しますが、今回は「few-shot」の例（メールと適切な分類（およびフォーマット））を使用してClaudeに正しい答えを出力させるように`PROMPT`を編集します。Claudeの出力の*最後*の文字がカテゴリーの文字になるようにしたいと思います。

各メールに対して正しい文字カテゴリーを忘れた場合は、`EMAILS`リストの横にあるコメントを参照してください。

メールのカテゴリーは以下の通りです：										
- (A) 購入前の質問
- (B) 破損または欠陥のある商品
- (C) 請求に関する質問
- (D) その他（説明してください）								

In [ ]:
# 変数コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = """このメールを緑または青に分類してください：{email}"""

# Claudeの応答の事前入力
PREFILL = ""

# リストとして保存された変数コンテンツ
EMAILS = [
    "こんにちは - 私のMixmaster4000を操作すると奇妙な音がします。また、電子機器が焼けるような、煙っぽいプラスチックのような臭いがします。交換が必要です。", # (B) 破損または欠陥のある商品
    "Mixmaster 4000を使って塗料を混ぜることはできますか？それとも食品を混ぜるだけのものですか？", # (A) 購入前の質問 または (D) その他（説明してください）
    "キャンセルしてから4ヶ月も月額料金が終わるのを待っています！！何が起こっているんだ？？？", # (C) 請求に関する質問
    "ここにどうやって来たのかわかりません。コンピューターが苦手です。助けて。" # (D) その他（説明してください）
]

# 1つのメールに対して複数の正しい分類の可能性を考慮して、リストのリストとして保存された正しい分類
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# メールのリストを反復処理
for i,email in enumerate(EMAILS):
    
    # メールテキストをメールプレースホルダー変数に代入
    formatted_prompt = PROMPT.format(email=email)
   
    # Claudeの応答を取得
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Claudeの応答を評価
    grade = any([bool(re.search(ans, response[-1])) for ans in ANSWERS[i]])
    
    # Claudeの応答を表示
    print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
    print("ユーザーターン")
    print(formatted_prompt)
    print("\nアシスタントターン")
    print(PREFILL)
    print("\n------------------------------------- Claudeの応答 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 評価 ------------------------------------------")
    print("この演習は正しく解かれました：", grade, "\n\n\n\n\n\n")

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_7_1_hint)

まだ困っていますか？以下のセルを実行して解答例を確認してください。

In [ ]:
print(hints.exercise_7_1_solution)

### おめでとうございます！

ここまですべての演習を解いたなら、次の章に進む準備ができています。プロンプトエンジニアリングを楽しんでください！

---

## サンプルプレイグラウンド

ここは、このレッスンで示されたプロンプトの例を自由に試したり、プロンプトを微調整してClaudeの応答にどのような影響があるかを確認したりできる領域です。

In [ ]:
# プロンプト
PROMPT = "サンタさんはクリスマスにプレゼントを持ってきてくれますか？"

# Claudeの応答を表示
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = """「A」として話す次の行を書いて、会話を完成させてください。
Q: 歯の妖精は本当にいるの？
A: もちろんだよ、かわいい子。歯を包んで今夜枕の下に置いてね。朝になったら何か待っているかもしれないよ。
Q: サンタさんはクリスマスにプレゼントを持ってきてくれますか？"""

# Claudeの応答を表示
print(get_completion(PROMPT))

In [ ]:
# 変数コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = """シルバーミスト・ホローは魅力的な村で、並外れた個性を持つ人々の住処でした。
その中にはリアム・パテル博士がいました。彼は地域の医療センターで外科手術の技術を革新した脳神経外科医でした。
オリビア・チェンは革新的な建築家で、持続可能で息をのむような設計で村の景観を変えました。
地元の劇場では、プロの訓練を受けた音楽家であり作曲家であるイーサン・コバックスの魅惑的な交響曲が演奏されていました。
地元の食材に情熱を注ぐ独学のシェフ、イザベラ・トーレスは、ファームトゥテーブルのレストランで料理の sensation を生み出し、食通にとって必須の訪問先となりました。
これらの素晴らしい個性を持つ人々は、それぞれ独特の才能を持ち、シルバーミスト・ホローの生活の鮮やかなタペストリーに貢献しました。
<individuals>
1. リアム・パテル博士 [脳神経外科医]
2. オリビア・チェン [建築家]
3. イーサン・コバックス [音楽家および作曲家]
4. イザベラ・トーレス [シェフ]
</individuals>

町の中心部では、シェフのオリバー・ハミルトンがファームトゥテーブルのレストラン、グリーン・プレートで料理シーンを変革しました。地元の有機食材にこだわるオリバーの姿勢は、食評家や地元の人々から絶賛を浴びています。
通りを少し下ったところには、リバーサイド・グローブ図書館があります。ここで主任司書のエリザベス・チェンは、すべての人にとって居心地の良い包括的な空間を作るために熱心に働いてきました。図書館の蔵書を増やし、子供向けの読書プログラムを確立する彼女の努力は、町の識字率に大きな影響を与えました。
魅力的な町の広場を歩くと、壁を飾る美しい壁画に魅了されるでしょう。これらの傑作は、有名なアーティスト、イザベラ・トーレスの作品です。彼女のリバーサイド・グローブの本質を捉える才能は、町に生命を吹き込みました。
リバーサイド・グローブの運動競技の成果も注目に値します。これは元オリンピック選手からコーチに転身したマーカス・ジェンキンスのおかげです。マーカスは自身の経験と情熱を活かして町の若者を指導し、リバーサイド・グローブ水泳チームを何度も地域選手権に導きました。
<individuals>
1. オリバー・ハミルトン [シェフ]
2. エリザベス・チェン [司書]
3. イザベラ・トーレス [アーティスト]
4. マーカス・ジェンキンス [コーチ]
</individuals>

オーク・バレーは魅力的な小さな町で、その技術と献身によってコミュニティに長続きする影響を与えた3人の注目すべき個人の住処です。
町の賑やかなファーマーズマーケットでは、情熱的な有機農家のローラ・シモンズを見つけることができます。彼女は美味しくて持続可能な方法で栽培された農産物で知られています。健康的な食生活を推進する彼女の献身は、町がよりエコ意識の高いライフスタイルを受け入れるよう促しました。
オーク・バレーのコミュニティセンターでは、熟練したダンスインストラクターのケビン・アルバレスが、あらゆる年齢層の人々に動きの喜びをもたらしました。彼の包括的なダンスクラスは、住民の間に団結と自己表現の感覚を育み、地元のアートシーンを豊かにしました。
最後に、tireless なボランティアのレイチェル・オコナーは、様々な慈善活動に時間を捧げています。他者の生活を改善することへの彼女のコミットメントは、オーク・バレー内に強いコミュニティ意識を作り出すのに不可欠でした。
それぞれのユニークな才能と揺るぎない献身を通じて、ローラ、ケビン、レイチェルはオーク・バレーの織物に自らを織り込み、活気に満ちた繁栄する小さな町を作り出すのを助けました。"""

# Claudeの応答の事前入力
PREFILL = "<individuals>"

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print("ユーザーターン:")
print(PROMPT)
print("\nアシスタントターン:")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))